# Saving data

In [1]:
#Import classic modules
# import numpy as np
# import matplotlib.pyplot as plt
import time


import qcodes as qc
from qcodes.logger.logger import start_all_logging
from qcodes.dataset.plotting import plot_dataset

import datetime
import numpy as np
import matplotlib.pyplot as plt
from time import sleep


%matplotlib inline
#%matplotlib notebook

In [2]:
#Redpitaya

# To import drivers from the folder 'C:\QCodes drivers and scripts\Drivers'
import sys
sys.path.append('C:/QCodes drivers and scripts/Drivers')

#Import the RedPitaya drivers
from redpitaya_qcodes_test import Redpitaya
#from qcodes.instrument_drivers.RP_test.redpitaya_qcodes2 import Redpitaya
RP = Redpitaya('Redpitaya', 'TCPIP::192.168.0.105::5001::SOCKET')

Connected to: REDPITAYA_IQ INSTR2018 (serial:0, firmware:01-02) in 0.30s


In [3]:
#Microwave source
from RS_SMB_100A import SMB100A

MW_source = SMB100A('SMB100A', 'TCPIP0::192.168.0.53::inst0::INSTR')#53 is the good one
#MW_source_2 = SMB100A('SMB100A', 'TCPIP0::192.168.0.54::inst0::INSTR')

Connected to: Rohde&Schwarz SMB100A (serial:1406.6000k03/177960, firmware:3.1.18.2-3.01.203.32) in 0.02s



## Creation of the Database

In [4]:
start_all_logging()

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\nicolas.roch\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\nicolas.roch\.qcodes\logs\200415-23452-qcodes.log


In [5]:
#create a station
#Error because of the new classes in the driver, but it's not too important
station = qc.Station()
station.add_component(RP)
station.add_component(MW_source)
station.snapshot()
station.components

2020-04-15 09:38:44,895 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 214 ¦ [Redpitaya(Redpitaya)] Snapshot: Could not update parameter: format_output
2020-04-15 09:38:44,895 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 214 ¦ [Redpitaya(Redpitaya)] Snapshot: Could not update parameter: nb_measure
2020-04-15 09:38:45,156 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 214 ¦ [Redpitaya(Redpitaya)] Snapshot: Could not update parameter: data_output
2020-04-15 09:38:45,156 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 214 ¦ [Redpitaya(Redpitaya)] Snapshot: Could not update parameter: pulse_zero
2020-04-15 09:38:45,156 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 214 ¦ [Redpitaya(Redpitaya)] Snapshot: Could not update parameter: length_time
2020-04-15 09:38:45,156 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 214 ¦ [Redpitaya(Redpitaya)] Snapshot: Could not update parameter: pulse_axis
2020-04-15 09:38:45,156

{'Redpitaya': <Redpitaya: Redpitaya>, 'SMB100A': <SMB100A: SMB100A>}

In [6]:
#define the name of the database in a sensible way
user = 'VJ'
date = str(datetime.date.today())
# date='2020-02-29'
description = 'Calibration_Redpitaya_test'#'Through-device'#'STWPA_V03_10'
database_name = date+"_"+user+"_"+description

#Create or Initialize the database
qc.initialise_or_create_database_at("C:/Data_QCodes/"+database_name)

#We can ask "where is the database?"
qc.config.core.db_location

Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]
Upgrading database; v1 -> v2: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 100.36it/s]
Upgrading database; v2 -> v3: : 0it [00:00, ?it/s]
Upgrading database; v3 -> v4: : 0it [00:00, ?it/s]
Upgrading database; v4 -> v5: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Upgrading database; v5 -> v6: : 0it [00:00, ?it/s]
Upgrading database; v7 -> v8: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


'C:/Data_QCodes/2020-04-15_VJ_Calibration_Redpitaya_test'

In [7]:
RP.print_readable_snapshot(update=False)

Redpitaya:
	parameter        value
--------------------------------------------------------------------------------
ADC1              :	Not available (V)
ADC2              :	Not available (V)
I1                :	Not available (V)
I1_INT            :	Not available (V)
I2                :	Not available (V)
I2_INT            :	Not available (V)
IDN               :	{'vendor': 'REDPITAYA_IQ', 'model': 'INSTR2018', 'serial'...
Q1                :	Not available (V)
Q1_INT            :	Not available (V)
Q2                :	Not available (V)
Q2_INT            :	Not available (V)
RESET             :	{} 
data_output       :	None 
data_size         :	1250 
decimation_filter :	0 
format_output     :	None 
freq_filter       :	2e+06 (Hz)
length_time       :	None 
mode_output       :	ADC 
nb_measure        :	None 
period            :	0.04 (s)
pulse_axis        :	None (NA)
pulse_zero        :	None 
start_ADC         :	0 (s)
status            :	None 
stop_ADC          :	1e-05 (s)
stop_DAC          :	6.5

## Reset

Here we just reset most of the parameters to avoid errors.

In [8]:
#Reset of the data_output
RP.RESET()

'{}'

In [9]:
#Just to ensure we are not generating any signal from the redpitaya
RP.stop_DAC(0)
RP.stop_DAC()

0.0

In [10]:
#We reset all the look-up tables
RP.reset_LUT()

## Parameter initialization

### Microwave source

In [11]:
#Pump frequency
w_p = 5e9 #Hz

#Intermediate frequency
w_if = 30e6

#Local oscillator frequency
w_lo = w_p - w_if

#Power

Pow = 5 #dBm

#Phase

phase = 0

#Individually uncomment the commands below to check the values.

MW_source.frequency(w_lo)#In Hz
#MW_source.frequency()

MW_source.power(Pow)#In dBm
#MW_source.power()

MW_source.phase(phase)
#MW_source.phase()

MW_source.status('on')
#MW_source.status('off')
MW_source.status()

'on'

In [12]:
#If we call RP.nb_measure(N) before this, it doesn't work.

t = 0.04 #Pulse period

RP.period(t) # maximum period is 1 s
type(RP.period())

print (RP.period(), 's')

0.04 s


### Acquisition parameters

In [13]:
# RP.status('stop')
# sleep(2)
# RP.status('start')
# sleep(2)

t_init = 0#300e-9
t_final = 10e-6

T = int(round(t_final/8e-9))-int(round(t_init/8e-9))

N_Pulse = 10000

RP.nb_measure(N_Pulse)
print('# of pulses',int(RP.nb_measure()))

RP.freq_filter(2e6)

# mode = 'ADC'
# RP.mode_output(mode)#('IQCH2')#'ADC', 'IQINT' , 'IQLP1'
# RP.mode_output()

RP.start_ADC(t_init)
RP.stop_ADC(t_final)


RP.format_output('ASCII')

# of pulses 10000


In [14]:
mode = 'ADC'
RP.mode_output(mode)#('IQCH2')#'ADC', 'IQINT' , 'IQLP1'
RP.mode_output()

#RP.data_output()

'ERR!ERR!ERR!ERR!ERR!ADC'

In [15]:
#To be modified in the future
RP.pulse_zero(0)
RP.pulse_zero()

0

In [16]:
RP.length_time(T*N_Pulse)
RP.length_time()

12500000

In [17]:
RP.time_axis()

array([0.00000000e+00, 1.00000008e+00, 2.00000016e+00, ...,
       1.24999980e+07, 1.24999990e+07, 1.25000000e+07])

In [18]:
RP.RESET()

'ERR!ERR!ERR!ERR!ERR!ERR!ERR!ERR!ERR!{}'

In [19]:
#RP.ADC1()

In [20]:
#Create an experiment and save data 
t0 = time.time()
###################################################### Experiment
exp_name = 'Acquire_ADC_CH1 '
sample_name = 'w_p:' + str(w_p) + '-' + 'w_if:' + str(w_if) + '-' + 'Pow:' + str(Pow) + '-' + 'Phase:' + str(phase) + '-' + 'Period:' + str(t) + '-' + 'IntLimit:' + str(t_init) + '_' + str(t_final) + '-' + 'N_Pulse:' + str(N_Pulse)
exp=qc.load_or_create_experiment(experiment_name=exp_name, sample_name=sample_name)
##############################################################


# mode = 'ADC'
# RP.mode_output(mode)
#time.sleep(0.2)
#meas = qc.Measurement(exp=exp, station=station)
#meas.register_parameter(vna.channels.S21.trace_mag_phase)


meas = qc.Measurement(exp=exp, station=station)
meas.register_parameter(RP.ADC1)
mode = 'ADC'
RP.mode_output(mode)
with meas.run() as datasaver:
    datasaver.add_result((RP.time_axis, RP.time_axis()),
                         (RP.ADC1, RP.ADC1()))
dataset = datasaver.dataset

t1 = time.time()

print('Time elapsed : ', t1-t0)

Starting experimental run with id: 1. 
10000 pulses. Mode: ADC
Time elapsed :  492.2476511001587


In [21]:
dataset.add_metadata('w_p',w_p)
dataset.add_metadata('w_if',w_if)
dataset.add_metadata('pow',Pow)
dataset.add_metadata('phase',phase)
dataset.add_metadata('Period',t)
dataset.add_metadata('t_init',t_init)
dataset.add_metadata('t_final',t_final)
dataset.add_metadata('N_Pulse',N_Pulse)